Author(s): Piyush Amitabh

Details: this code generates mip and stitches them to visualize the tracks

Created: May 02, 2023

License: GNU GPL v3.0

Comment: it is scope agnostic (KLA/WIL LSM) and os agnostic

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# from scipy import ndimage as ndi
# import pickle
import skimage
from PIL import Image
import tifffile as tiff
from natsort import natsorted

In [28]:
from tqdm import tqdm
# import plotly.express as px
# import plotly.graph_objects as go
# import dask
import configparser

In [5]:
#ask the user for the inputs

print('Warning: This code ONLY works with single channel z-stack tiff images. It will give unpredictable results with multiple channels.')
print('Warning: This code assumes the image name has channel name and position/region name for stiching')
main_dir = input('Enter the Parent directory where ALL the image stacks are stored: ')

# Batchprocess MIP

In [8]:
channel_names = ['GFP', 'RFP']

for root, subfolders, filenames in os.walk(main_dir):
    for filename in filenames:
        # print(f'Reading: {filename}')
        
        filepath = os.path.join(root, filename)
        # print(f'Reading: {filepath}')

        filename_list = filename.split('.')
        og_name = filename_list[0] #first of list=name
        ext = filename_list[-1] #last of list=extension

        if ext=="tif" or ext=="tiff": #tiff files
            print(f'Reading Image: {filepath}')
            read_image = tiff.imread(filepath)

            if len(read_image.shape)!=2: #image stacks, create MIP
                arr_mip = np.max(read_image, axis=0)

            for ch_name in channel_names:
                if ch_name.casefold() in og_name.casefold():
                    dest = os.path.join(root, ch_name.casefold()+'_mip')
                    if not os.path.exists(dest): #check if the dest exists
                        print("Write path doesn't exist.")
                        os.makedirs(dest)
                        print(f"Directory '{ch_name.casefold()}'_mip created")
                img_mip = Image.fromarray(arr_mip)
                img_mip.save(os.path.join(dest, og_name+'_mip.tif'))
                

Reading Image: /mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/GFP/Timepoint20_Pos3_GFP_ds.ome.tif
LSM Scope used: KLA
Downscaling factor = 4
Fatal Error: Exiting
Reading Image: /mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/GFP/Timepoint76_Pos1_GFP_ds.ome.tif
LSM Scope used: KLA
Downscaling factor = 4
ERROR: Can't determine LSM scope automatically.
Enter manually


ValueError: invalid literal for int() with base 10: ''

: 

# Stitching

In [6]:
print("Instructions for stitching:")
print("Image stitching works by reading stage positions from the 'notes.txt' file generated during acquisition")
print("-Images MUST have 'timepoint' substring in their names")
print("-Images MUST have 'pos' or 'region' substring in their names")
print("-Images MUST have channel substring(BF/GFP/RFP) in their names")

stitch_dir_flag = input("Is stitching directory same as Parent directory above?(y/n): ")
if stitch_dir_flag.casefold()=='y':
    stitch_dir = main_dir
else:
    stitch_dir = input('Enter the location of Parent directory for stitching (must contain MIPs and notes.txt file inside): ')

Instructions for stitching:
Image stitching works by reading stage positions from the 'notes.txt' file generated during acquisition
-Images MUST have 'timepoint' substring in their names
-Images MUST have 'pos' or 'region' substring in their names
-Images MUST have channel substring(BF/GFP/RFP) in their names


In [ ]:
global findscope_flag
findscope_flag = 0

def find_lsm_scope(img_h, img_w):
    '''Finds LSM Scope and downscaling factor automatically using image height and width.
    Returns: 
    ds_factor_h = downscaling factor in height, 
    ds_factor_w = downscaling factor in width'''

    if img_w==img_h: # probably KLA LSM
        findscope_flag = 1

        ds_factor_h = 2048//img_h
        ds_factor_w = 2048//img_w
        r = 2048%img_h

        if r>0: #implying downscaling factor is in fraction
            findscope_flag = 0
            print("Downscaling factor in fraction. Can't process automatically.")

    elif img_w>img_h: # probably WIL LSM
        findscope_flag = 2

        ds_factor_h = 2160//img_h
        ds_factor_w = 2560//img_w

        if ds_factor_h!=ds_factor_w:
            findscope_flag=0

        r_h = 2160%img_h
        r_w = 2560%img_w
        
        if r_h>0 or r_w>0 : #implying downscaling factor is in fraction
            findscope_flag = 0
            print("Downscaling factor in fraction. Can't process automatically.")

    if findscope_flag==1:
        print('LSM Scope used: KLA')
        print(f'Downscaling factor = {ds_factor_w}')
    elif findscope_flag==2:
        print('LSM Scope used: WIL')
        print(f'Downscaling factor = {ds_factor_w}')

    user_check = input('Is the above information correct?(y/n): ')
    if user_check.casefold()=='n':
        findscope_flag = 0

    if findscope_flag==0: #couldn't find scope, enter manually
        print("ERROR: Failed to determine LSM scope automatically.\nEnter manually")
        findscope_flag = int(input('Enter the scope used:\n1 - KLA LSM Scope\n2 - WIL LSM Scope\nInput (1/2): '))
        if findscope_flag==1 or findscope_flag==2:
            ds_factor_h = int(input('Enter the downscaling factor in height: '))
            ds_factor_w = int(input('Enter the downscaling factor in width: '))
        else:
            print("Fatal Error: Exiting")
            exit()

    return(ds_factor_h, ds_factor_w)

In [ ]:
pos_max = input('Enter number of positions/regions of imaging per timepoint: ')

In [14]:
stitch_dir

'/mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/'

In [24]:
#make a list of all img files by channel for stitching
sub_names = ['BF', 'GFP', 'RFP']
bf_flag = False #0 means not found, 1 mean found
gfp_flag = False
rfp_flag = False


for root, subfolders, filenames in os.walk(stitch_dir):
    for filename in filenames:
        # print(f'Reading: {filename}')
        
        filepath = os.path.join(root, filename)
        # print(f'Reading: {filepath}')

        filename_list = filename.split('.')
        og_name = filename_list[0] #first of list=name
        ext = filename_list[-1] #last of list=extension

        if (ext=="tif" or ext=="tiff"):
            if (not bf_flag) and ('bf' in og_name.casefold()): #find BF
                print('BF images found at:'+root)
                bf_path = root
                bf_img_list = natsorted(os.listdir(root))
                bf_flag = True
            elif 'mip' in og_name.casefold():
                if (not gfp_flag) and ('gfp' in og_name.casefold()):
                    print('GFP MIP images found at:'+root)
                    gfp_mip_path = root
                    gfp_img_list = natsorted(os.listdir(root))
                    gfp_flag = True
                elif (not rfp_flag) and ('rfp' in og_name.casefold()):
                    print('RFP MIP images found at:'+root)
                    rfp_mip_path = root
                    rfp_img_list = natsorted(os.listdir(root))
                    rfp_flag = True

BF images found at:/mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/BF
GFP MIP images found at:/mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/GFP/gfp_mip


In [35]:
#find the nearest notes.txt
config = configparser.ConfigParser()

if gfp_flag:
    start_path = gfp_mip_path
elif rfp_flag:
    start_path = rfp_mip_path
elif bf_flag:
    start_path = bf_path

target1 = 'notes.txt'
target2 = 'Notes.txt'
while True:
    if os.path.isfile(os.path.join(start_path,target1)):
        #found
        print(f'found {target1} at:'+start_path)
        config.read(os.path.join(start_path,target1))
        break
    elif os.path.isfile(os.path.join(start_path,target2)):
        #found
        print(f'found {target2} at:'+start_path)
        config.read(os.path.join(start_path,target2))
        break

    if os.path.dirname(start_path)==start_path: #reached root
        #not found
        print("Error: Can't find notes.txt, Enter manually")
        break    
    start_path=os.path.dirname(start_path)

found Notes.txt at:/mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2


In [36]:
config.sections()

['COMMENTS',
 'Spim Galvo Settings',
 'Acquisition Settings',
 'Fish 1 Notes',
 'Fish 1 Region 1',
 'Fish 1 Region 2',
 'Fish 1 Region 3',
 'Fish 1 Region 4',
 'Fish 1 Region 5',
 'Fish 2 Notes',
 'Fish 2 Region 1',
 'Fish 2 Region 2',
 'Fish 2 Region 3',
 'Fish 2 Region 4',
 'Fish 2 Region 5']

In [37]:
config.getfloat("Fish 1 Region 1", "x_position")

-8168.0

---

In [ ]:
sub_names = ['BF', 'GFP', 'RFP']

for root, subfolders, filenames in os.walk(stitch_dir):
    for filename in filenames:
        # print(f'Reading: {filename}')
        
        filepath = os.path.join(root, filename)
        # print(f'Reading: {filepath}')

        filename_list = filename.split('.')
        og_name = filename_list[0] #first of list=name
        ext = filename_list[-1] #last of list=extension

        if (ext=="tif" or ext=="tiff") and ('mip' in og_name.casefold() or 'bf' in og_name.casefold()): #tiff files

            for sub in sub_names:
                if sub.casefold() in og_name.casefold():
                    dest = os.path.join(root, sub.casefold()+'_stitched')
                    if not os.path.exists(dest): #check if the dest exists
                        print("Write path doesn't exist.")
                        os.makedirs(dest)
                        print(f"Directory '{sub.casefold()}_stitched' created")

            #read all mips in this location


            #get info from the notes.txt

            #read all images per timepoint then stitch and save them at dest 
            

            print(f'Reading Image: {filepath}')
            read_image = tiff.imread(filepath)


            
            if len(read_image.shape)==2: #single images, assume BF
                if findscope_flag==0: #skimage coords is (row, col)
                    ds_h, ds_w = find_lsm_scope(img_h = read_image.shape[0], img_w = read_image.shape[1])

            elif len(read_image.shape)!=2: #image stacks, create MIP
                if findscope_flag==0: #skimage coords is (plane, row, col)
                    ds_h, ds_w = find_lsm_scope(img_h = read_image.shape[1], img_w = read_image.shape[2])

                #generate max intensity zproject from stack_full
                arr_mip = np.max(read_image, axis=0)


                # os.chdir(dest)
                img_mip = Image.fromarray(arr_mip)
                img_mip.save(os.path.join(dest, og_name+'_mip.tif'))
                

Reading Image: /mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/GFP/Timepoint20_Pos3_GFP_ds.ome.tif
LSM Scope used: KLA
Downscaling factor = 4
Fatal Error: Exiting
Reading Image: /mnt/HDD/Demeter/Piyush_hedonismBot/time lapse_conv_downsampled/time lapse_10_05_22_night/Fish2/GFP/Timepoint76_Pos1_GFP_ds.ome.tif
LSM Scope used: KLA
Downscaling factor = 4
ERROR: Can't determine LSM scope automatically.
Enter manually


ValueError: invalid literal for int() with base 10: ''

: 

In [1]:
# The pixel spacing in our LSM image is 1µm in the z axis, and  0.1625µm in the x and y axes.

zd, xd, yd = 1, 0.1625, 0.1625 #zeroth dimension is z in skimage coords
orig_spacing = np.array([zd, xd, yd]) #change to the actual pixel spacing from the microscope
new_spacing = np.array([zd, xd*ds_h, yd*ds_w]) #downscale x&y by n

NameError: name 'np' is not defined